## Imports

In [ ]:
!pip list | grep -E "pandas|numpy|scikit-learn|joblib"

geopandas                                1.1.1
joblib                                   1.5.2
numpy                                    2.0.2
pandas                                   2.2.2
pandas-datareader                        0.10.0
pandas-gbq                               0.30.0
pandas-stubs                             2.2.2.240909
scikit-learn                             1.6.1
sklearn-pandas                           2.2.0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
import joblib

## Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Projetos/data"
df_train = pd.read_csv(f'{path}/train.csv')
df_test = pd.read_csv(f'{path}/test.csv')

In [ ]:
df_train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [ ]:
df_test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [ ]:
df_train = df_train.drop('ID_code', axis=1)

In [ ]:
X_train, y_train = df_train.drop('target', axis=1), df_train['target']

In [ ]:
all(df_train.isnull().sum()) == False

True

## Redução de dimensionalidade

In [ ]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

In [ ]:
X_reduced

array([[  5.60646899,  23.55440926,  -1.899704  , ...,  -5.41563294,
          1.98110756,  -5.4449445 ],
       [-34.87802792, -11.46929107,  -3.75977398, ...,   2.92756827,
         -0.47994981,   2.75461323],
       [  4.79023946,   0.33421344,  11.35931802, ...,  -1.1990427 ,
         -2.04303563,  -4.956015  ],
       ...,
       [-36.84227048,  -7.76008289,  30.23221393, ...,  -1.26350447,
          4.61521089,  -0.41376635],
       [-30.23091675,  24.4235161 ,  25.55355302, ...,  -5.882554  ,
         -0.32885376,  -2.80609371],
       [  5.99488377,  -7.69716402,  10.11166579, ...,   4.22976642,
         -1.09201667,  -1.02511155]])

In [ ]:
componentes = pca.components_
componentes

array([[-2.26160073e-04, -6.91690470e-04,  9.05051683e-05, ...,
         3.92273398e-05,  1.42440866e-04, -1.49602509e-03],
       [ 1.20429067e-03,  4.00187658e-04,  4.14065699e-04, ...,
         2.41310302e-05,  3.19419636e-05, -6.56170969e-03],
       [-4.54096083e-04, -1.50947043e-03,  3.77436487e-04, ...,
        -4.54178417e-05, -4.06134501e-04,  1.25216713e-03],
       ...,
       [ 1.63602771e-04, -2.41315992e-03, -3.19511427e-03, ...,
        -3.99571707e-04,  4.63417646e-03,  7.65252899e-04],
       [ 1.03474632e-02, -3.84011344e-03, -2.03423951e-03, ...,
         9.39567912e-04,  2.90030987e-03,  4.89678892e-04],
       [ 2.55473735e-02, -9.10805829e-03,  5.52564675e-03, ...,
        -1.42981250e-03,  1.41968769e-02, -3.37340615e-04]])

In [ ]:
joblib.dump(pca, f'{path}/pca_model.pkl')

['/content/drive/MyDrive/Projetos/data/pca_model.pkl']

## Treinamento

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_reduced, y_train)


### Previsão

In [ ]:
X_test = df_test.drop("ID_code", axis=1)
X_test_reduced = pca.transform(X_test)
clf.predict(X_test_reduced)

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
joblib.dump(clf, f'{path}/gbc_model.pkl') # exporta o modelo

['/content/drive/MyDrive/Projetos/data/gbc_model.pkl']